# **Bioinformatics VEGFR2 Project**

---

## **ChEMBL Database**

The [*ChEMBL Database*](https://www.ebi.ac.uk/chembl/) is a database that contains curated bioactivity data of more than 2.5 million compounds. It is compiled from more than 92,100 documents, 1.7 million assays and the data spans 16,000 targets and 2,100 cells and 48,800 indications.
[Data of May 07, 2025; ChEMBL version 35].

## **Installing libraries**

Install the ChEMBL web service package so that we can retrieve bioactivity data from the ChEMBL Database.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
! pip install chembl_webresource_client

## **Importing libraries**

In [ ]:
# Import necessary libraries
import pandas as pd
from chembl_webresource_client.new_client import new_client

### **Target search for VEGFR2**

In [ ]:
# Target search for coronavirus
target = new_client.target
target_query = target.search('CHEMBL279')
targets = pd.DataFrame.from_dict(target_query)
targets

:### **Select and retrieve bioactivity data for *VEGFR2***

---



We will assign the 1st entry (which corresponds to the target protein, *VEGFR2* to the ***selected_target*** variable

In [ ]:
selected_target = targets.target_chembl_id[0]
selected_target

Here, we will retrieve only bioactivity data for *VEGFR2* (CHEMBL279) that are reported as pChEMBL values.

In [ ]:
activity = new_client.activity
res = activity.filter(target_chembl_id=selected_target).filter(standard_type="IC50")

In [ ]:
df = pd.DataFrame.from_dict(res)

In [ ]:
df

## **Handling missing data**
If any compounds has missing value for the **standard_value** and **canonical_smiles** column then drop it.

In [ ]:
df2 = df[df.standard_value.notna()]
df2 = df2[df.canonical_smiles.notna()]
df2

In [ ]:
len(df2.canonical_smiles.unique())

In [ ]:
df2_nr = df2.drop_duplicates(['canonical_smiles'])
df2_nr

In [ ]:
print(type(df2_nr))

In [ ]:
df2_nr.to_csv('df2_nr.csv', index=False)

## **Data pre-processing of the bioactivity data**

### **Combine the 3 columns (molecule_chembl_id,canonical_smiles,standard_value) and bioactivity_class into a DataFrame**

In [ ]:
selection = ['molecule_chembl_id','canonical_smiles','standard_value']
df3 = df2_nr[selection]
df3

Saves dataframe to CSV file

In [ ]:
df3.to_csv('df3.csv', index=False)

### **Labeling compounds as either being active, or inactive**
The bioactivity data is in the IC50 unit. Compounds having values of less than 100 nM will be considered to be **active** while those greater than 1,000 nM will be considered to be **inactive**.

In [ ]:
df4 = pd.read_csv('df3.csv')

In [ ]:
df4

In [ ]:
bioactivity_threshold = []
for i in df4.standard_value:
  if float(i) >= 100:
    bioactivity_threshold.append("inactive")
  else float(i) <= 100:
    bioactivity_threshold.append("active")

In [ ]:
bioactivity_class = pd.Series(bioactivity_threshold, name='class')
df5 = pd.concat([df4, bioactivity_class], axis=1)
df5

Saves dataframe to CSV file

In [ ]:
df5.to_csv('df5.csv', index=False)

# **Exploratory Data Analysis**
---

## **Install conda and rdkit**

In [ ]:
! wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
! chmod +x Miniconda3-py37_4.8.2-Linux-x86_64.sh
! bash ./Miniconda3-py37_4.8.2-Linux-x86_64.sh -b -f -p /usr/local
! conda install -c rdkit rdkit -y
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

## **Load bioactivity data**

In [ ]:
df5_no_smiles = df4.drop(columns='canonical_smiles')

In [ ]:
df4_no_smiles

In [ ]:
smiles = []

for i in df4.canonical_smiles.tolist():
  cpd = str(i).split('.')
  cpd_longest = max(cpd, key = len)
  smiles.append(cpd_longest)

smiles = pd.Series(smiles, name = 'canonical_smiles')

In [ ]:
df5_clean_smiles = pd.concat([df4_no_smiles,smiles], axis=1)
df5_clean_smiles

In [ ]:
df1 = df5_clean_smiles.dropna()

In [ ]:
df1


## **Calculate Lipinski descriptors**
Christopher Lipinski, a scientist at Pfizer, came up with a set of rule-of-thumb for evaluating the **druglikeness** of compounds. Such druglikeness is based on the Absorption, Distribution, Metabolism and Excretion (ADME) that is also known as the pharmacokinetic profile. Lipinski analyzed all orally active FDA-approved drugs in the formulation of what is to be known as the **Rule-of-Five** or **Lipinski's Rule**.

The Lipinski's Rule stated the following:
* Molecular weight < 500 Dalton
* Octanol-water partition coefficient (LogP) < 5
* Hydrogen bond donors < 5
* Hydrogen bond acceptors < 10

### **Import libraries**

In [ ]:
!pip install rdkit-pypi


In [ ]:
from rdkit.Chem import Descriptors, Lipinski


### **Calculate descriptors**

In [ ]:
from rdkit import Chem
from rdkit.Chem import Descriptors, Lipinski
import numpy as np
import pandas as pd

def lipinski(smiles, verbose=False):
    moldata = []
    for elem in smiles:
        mol = Chem.MolFromSmiles(elem)
        moldata.append(mol)

    baseData = np.arange(1, 1)
    i = 0
    for mol in moldata:
        desc_MolWt = Descriptors.MolWt(mol)
        desc_MolLogP = Descriptors.MolLogP(mol)
        desc_NumHDonors = Lipinski.NumHDonors(mol)
        desc_NumHAcceptors = Lipinski.NumHAcceptors(mol)
        desc_NumRotatableBonds = Lipinski.NumRotatableBonds(mol)


        # Calculate TPSA
        desc_TPSA = Descriptors.TPSA(mol)

        row = np.array([desc_MolWt,
                        desc_MolLogP,
                        desc_NumHDonors,
                        desc_NumHAcceptors,
                        desc_NumRotatableBonds,
                        desc_TPSA  # Add TPSA to the array
                        ])

        if i == 0:
            baseData = row
        else:
            baseData = np.vstack([baseData, row])
        i += 1

    columnNames = ["MW", "LogP", "NumHDonors", "NumHAcceptors", "NumRotatableBonds", "TPSA"]
    descriptors = pd.DataFrame(data=baseData, columns=columnNames)

    return descriptors

In [ ]:
df_lipinski = lipinski(df.canonical_smiles)
df_lipinski

### **Combine DataFrames**

Let's take a look at the 2 DataFrames that will be combined.

In [ ]:
df_lipinski

Now, let's combine the 2 DataFrame

In [ ]:
df_combined = pd.concat([df,df_lipinski], axis=1)

In [ ]:
df_combined

In [ ]:
df_combined.to_csv('Lipinski.csv')

### **Convert IC50 to pIC50**
To allow **IC50** data to be more uniformly distributed, we will convert **IC50** to the negative logarithmic scale which is essentially **-log10(IC50)**.

This custom function pIC50() will accept a DataFrame as input and will:
* Take the IC50 values from the ``standard_value`` column and converts it from nM to M by multiplying the value by 10$^{-9}$
* Take the molar value and apply -log10
* Delete the ``standard_value`` column and create a new ``pIC50`` column

In [ ]:
import numpy as np

def pIC50(input):
    pIC50 = []

    for i in input['standard_value_norm']:
        molar = i*(10**-9) # Converts nM to M
        pIC50.append(-np.log10(molar))

    input['pIC50'] = pIC50
    x = input.drop('standard_value_norm', axis=1)

    return x


Point to note: Values greater than 100,000,000 will be fixed at 100,000,000 otherwise the negative logarithmic value will become negative.

In [ ]:
df_combined.standard_value.describe()

In [ ]:
-np.log10( (10**-9)* 100000000 )

In [ ]:
-np.log10( (10**-9)* 10000000000 )

We will first apply the norm_value() function so that the values in the standard_value column is normalized.

In [ ]:

def norm_value(input):
    norm = []

    for i in input['standard_value']:
        if i > 100000000:
          i = 100000000
        norm.append(i)

    input['standard_value_norm'] = norm
    x = input.drop('standard_value', axis=1)

    return x

In [ ]:
df_norm = norm_value(df_combined)
df_norm

In [ ]:
df_norm.standard_value_norm.describe()

In [ ]:
df_final = pIC50(df_norm)
df_final

In [ ]:
df_final.to_csv('VEGFR2_pIC50.csv')

In [ ]:
df_final.pIC50.describe()

## **Exploratory Data Analysis (Chemical Space Analysis) via Lipinski descriptors**

### **Import library**

In [ ]:
import pandas as pd

### **Frequency plot of the 2 bioactivity classes**

In [ ]:
df_final=pd.read_csv('/content/df_final_2classes.csv')

In [ ]:
df_final

In [ ]:
import seaborn as sns
sns.set(style='ticks')
import matplotlib.pyplot as plt
import pandas as pd # Make sure pandas is imported if not already

# Load the dataframe (assuming this was done in a previous cell)
# If not, uncomment the line below:
# df_final=pd.read_csv('chembl_zbi_classification.csv')


plt.figure(figsize=(8, 6))

# Assuming 'bioactivity_class' is the correct column name based on previous steps
sns.countplot(x='bioactivity_class', data=df_final, hue='bioactivity_class', edgecolor='black', palette=['#feb236', '#6b5b95'])

plt.xlabel('Bioactivity class', fontsize=14, fontweight='bold')
plt.ylabel('Frequency', fontsize=14, fontweight='bold')

# Save the plot as an SVG file
plt.savefig('plot_bioactivity_class.svg')

# Optionally, you can display the plot as well
# plt.show()

### **Scatter plot of MW versus LogP**

It can be seen that the 2 bioactivity classes are spanning similar chemical spaces as evident by the scatter plot of MW vs LogP.

In [ ]:
import numpy as np # Ensure numpy is imported for np.nan and np.ndarray
import pandas as pd # Ensure pandas is imported
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(8, 6))
sns.scatterplot(x='MW', y='LogP', data=df_final, hue='bioactivity_class', size='pIC50', edgecolor='black', alpha=0.5, palette=['#feb236', '#6b5b95'])

plt.xlabel('MW in g/mol', fontsize=14, fontweight='bold', fontstyle='normal')
plt.ylabel('cLogP', fontsize=14, fontweight='bold', fontstyle='normal')
plt.legend(bbox_to_anchor=(0.01, 1), loc=2, borderaxespad=0, fontsize=9)
plt.savefig('plot_MW_vs_LogP.svg')

### **Box plots**

#### **pIC50 value**

In [ ]:
plt.figure(figsize=(8, 6))

sns.boxplot(x = 'bioactivity_class', y = 'pIC50', hue='bioactivity_class', data = df_final, palette=['#feb236', '#6b5b95'])

plt.xlabel('Bioactivity class', fontsize=14, fontweight='bold')
plt.ylabel('pIC50 value', fontsize=14, fontweight='bold')

plt.savefig('plot_ic50.svg')

**Statistical analysis | Mann-Whitney U Test**

In [ ]:
def mannwhitney(descriptor, verbose=False):
  # https://machinelearningmastery.com/nonparametric-statistical-significance-tests-in-python/
  from numpy.random import seed
  from numpy.random import randn
  from scipy.stats import mannwhitneyu

# seed the random number generator
  seed(1)

# actives and inactives
  selection = [descriptor, 'bioactivity_class']
  df = df_final[selection]
  active = df[df.bioactivity_class == 'active']
  active = active[descriptor]

  selection = [descriptor, 'bioactivity_class']
  df = df_final[selection]
  inactive = df[df.bioactivity_class == 'inactive']
  inactive = inactive[descriptor]

# compare samples
  stat, p = mannwhitneyu(active, inactive)
  #print('Statistics=%.3f, p=%.3f' % (stat, p))

# interpret
  alpha = 0.05
  if p > alpha:
    interpretation = 'Same distribution (fail to reject H0)'
  else:
    interpretation = 'Different distribution (reject H0)'

  results = pd.DataFrame({'Descriptor':descriptor,
                          'Statistics':stat,
                          'p':p,
                          'alpha':alpha,
                          'Interpretation':interpretation}, index=[0])
  filename = 'mannwhitneyu_' + descriptor + '.csv'
  results.to_csv(filename)

  return results

In [ ]:
mannwhitney('pIC50')

#### **MW**

In [ ]:
plt.figure(figsize=(8, 6))

sns.boxplot(x = 'bioactivity_class', y = 'MW', hue='bioactivity_class', data = df_final, palette=['#feb236', '#6b5b95'])

plt.xlabel('Bioactivity class', fontsize=14, fontweight='bold')
plt.ylabel('MW in g/mol', fontsize=14, fontweight='bold')

plt.savefig('plot_MW.svg')

In [ ]:
mannwhitney('MW')

#### **LogP**

In [ ]:
plt.figure(figsize=(8, 6))

sns.boxplot(x = 'bioactivity_class', y = 'LogP', hue='bioactivity_class', data = df_final, palette=['#feb236', '#6b5b95'])

plt.xlabel('Bioactivity class', fontsize=14, fontweight='bold')
plt.ylabel('cLogP', fontsize=14, fontweight='bold')

plt.savefig('plot_LogP.svg')

**Statistical analysis | Mann-Whitney U Test**

In [ ]:
mannwhitney('LogP')

#### **NumHDonors**

In [ ]:
plt.figure(figsize=(8, 6))

sns.boxplot(x = 'bioactivity_class', y = 'NumHDonors', hue='bioactivity_class', data = df_final, palette=['#feb236', '#6b5b95'])

plt.xlabel('Bioactivity class', fontsize=14, fontweight='bold')
plt.ylabel('nHD', fontsize=14, fontweight='bold')

plt.savefig('plot_NumHDonors.svg')

**Statistical analysis | Mann-Whitney U Test**

In [ ]:
mannwhitney('NumHDonors')

#### **NumHAcceptors**

In [ ]:
plt.figure(figsize=(8, 6))

sns.boxplot(x = 'bioactivity_class', y = 'NumHAcceptors', hue='bioactivity_class', data = df_final, palette=['#feb236', '#6b5b95'])

plt.xlabel('Bioactivity class', fontsize=14, fontweight='bold')
plt.ylabel('nHA', fontsize=14, fontweight='bold')


plt.savefig('plot_NumHAcceptors.svg')

In [ ]:
mannwhitney('NumHAcceptors')

In [ ]:
plt.figure(figsize=(8, 6))

sns.boxplot(x = 'bioactivity_class', y = 'TPSA', hue='bioactivity_class', data = df_final, palette=['#feb236', '#6b5b95'])

plt.xlabel('Bioactivity class', fontsize=14, fontweight='bold')
plt.ylabel('TPSA in Å²', fontsize=14, fontweight='bold')

plt.savefig('plot_TPSA.svg')

In [ ]:
mannwhitney('TPSA')

In [ ]:
plt.figure(figsize=(8, 6))

sns.boxplot(x = 'bioactivity_class', y = 'NumRotatableBonds', hue='bioactivity_class', data = df_final, palette=['#feb236', '#6b5b95'])

plt.xlabel('Bioactivity class', fontsize=14, fontweight='bold')
plt.ylabel('nRot', fontsize=14, fontweight='bold')

plt.savefig('plot_nRot.svg')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
mannwhitney('NumRotatableBonds')

# **BDescriptor Calculation and Dataset Preparation**
---

## **Download PaDEL-Descriptor**

In [ ]:
! apt-get update
! apt-get install -y openjdk-8-jre-headless

from padelpy import padeldescriptor

In [ ]:
! pip install padelpy

In [ ]:
! wget https://github.com/dataprofessor/padel/raw/main/fingerprints_xml.zip
! unzip fingerprints_xml.zip

In [ ]:
import glob
xml_files = glob.glob("*.xml")
xml_files.sort()
xml_files

In [ ]:
FP_list = ['AtomPairs2DCount',
 'AtomPairs2D',
 'EState',
 'CDKextended',
 'CDK',
 'CDKgraphonly',
 'KlekotaRothCount',
 'KlekotaRoth',
 'MACCS',
 'PubChem',
 'SubstructureCount',
 'Substructure']

In [ ]:
fp = dict(zip(FP_list, xml_files))
fp

In [ ]:
selection = ['smiles','ID']
df3_selection = df3[selection]
df3_selection.to_csv('molecule.smi', sep='\t', index=False, header=False)

In [ ]:
! cat molecule.smi | head -5

In [ ]:
! cat molecule.smi | wc -l

In [ ]:
descriptors = pd.read_csv(fingerprint_output_file)
descriptors

## **Calculate fingerprint descriptors**


In [ ]:
fingerprint = 'PubChem'

fingerprint_output_file = ''.join([fingerprint,'.csv']) #Substructure.csv
fingerprint_descriptortypes = fp[fingerprint]

padeldescriptor(mol_dir='molecule.smi',
                d_file=fingerprint_output_file, #'Substructure.csv'
                #descriptortypes='PubchemFingerprinter.xml',
                descriptortypes= fingerprint_descriptortypes,
                detectaromaticity=True,
                standardizenitro=True,
                standardizetautomers=True,
                threads=2,
                removesalt=True,
                log=True,
                fingerprints=True)

In [ ]:
fp['PubChem']

### **Calculate PaDEL descriptors**

In [ ]:
df3_X = pd.read_csv('descriptors_output.csv')

## **Preparing the X and Y Data Matrices**

### **X data matrix**

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df=pd.read_csv('/content/mordred_descriptors.csv')

In [ ]:
df.columns

In [ ]:
df_7 = df.drop(columns=['compounds', 'MW', 'pIC50', 'Groups', 'LogP','NumHDonors', 'NumHAcceptors', 'NumRotatableBonds', 'TPSA'])

### **3.4. Remove high correlation columns and low variance features**

In [ ]:
import pandas as pd
from sklearn.feature_selection import VarianceThreshold
if not isinstance(df_7, pd.DataFrame):
    df_7 = pd.DataFrame(df_7)

# Correlation filtering
correlation_matrix = df_7.corr()
high_corr_columns = set()
for i in range(len(correlation_matrix.columns)):
    for j in range(i):
        if abs(correlation_matrix.iloc[i, j]) > 0.90:
            colname = correlation_matrix.columns[i]
            high_corr_columns.add(colname)

df_7_filtered = df_7.drop(columns=high_corr_columns)
print(f"Removed columns: {high_corr_columns}")
print(f"Retained columns after correlation filtering: {df_7_filtered.columns.tolist()}")

# Variance thresholding
selection = VarianceThreshold(threshold=(.8 * (1 - .8)))
X = selection.fit_transform(df_7_filtered)
retained_columns_variance = df_7_filtered.columns[selection.get_support()].tolist()
print(f"Retained columns after variance thresholding: {retained_columns_variance}")
print(f"Shape of transformed data: {X.shape}")

In [ ]:
X.to_csv('X.csv', index=True)

In [ ]:
Y=df['Groups']

## **4. Data split (80/20 ratio)**

In [ ]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
X, Y = shuffle(X, Y, random_state=42)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
X_train.shape, Y_train.shape

In [ ]:
X_test.shape, Y_test.shape

# **Model Building**

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, ParameterGrid
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef, roc_auc_score
param_grid = {"n_neighbors": [3, 5, 7, 9],"weights": ["uniform", "distance"],"p": [1, 2]}
grid = list(ParameterGrid(param_grid))
n_repeats = 10
metrics_list = []
for i in range(n_repeats):
    print(f"Running repetition {i+1}/{n_repeats} ...")
    X_train, X_test, Y_train, Y_test = train_test_split(
        X, y,
        test_size=0.2,
        stratify=y,
        random_state=100 + i
    )
    best_auc = -1
    best_model = None

    for params in grid:
        knn = KNeighborsClassifier(**params)
        knn.fit(X_train, Y_train)
        y_prob = knn.predict_proba(X_test)[:, 1]
        auc = roc_auc_score(Y_test, y_prob)

        if auc > best_auc:
            best_auc = auc
            best_model = knn
    y_pred = best_model.predict(X_test)
    y_prob = best_model.predict_proba(X_test)[:, 1]
    metrics_list.append({
        "Accuracy": accuracy_score(Y_test, y_pred),
        "Precision": precision_score(Y_test, y_pred),
        "Recall": recall_score(Y_test, y_pred),
        "F1": f1_score(Y_test, y_pred),
        "MCC": matthews_corrcoef(Y_test, y_pred),
        "AUC": roc_auc_score(Y_test, y_prob),
    })
    best_params_list.append(best_params)
results_df = pd.DataFrame(metrics_list)
summary = results_df.agg(['mean', 'std']).T
best_params_df = pd.DataFrame(best_params_list)
print(summary)
print(best_params_df)



In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, ParameterGrid
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef, roc_auc_score
param_grid = {"C": [0.01, 0.1, 1, 10],"penalty": ["l2"],"solver": ["lbfgs"]}
grid = list(ParameterGrid(param_grid))
n_repeats = 10
metrics_list = []
for i in range(n_repeats):
    print(f"Running repetition {i+1}/{n_repeats} ...")
    X_train, X_test, Y_train, Y_test = train_test_split(
        X, y,
        test_size=0.2,
        stratify=y,
        random_state=100 + i
    )

    best_auc = -1
    best_model = None
    for params in grid:
        lr = LogisticRegression(**params, max_iter=1000)
        lr.fit(X_train, Y_train)
        y_prob = lr.predict_proba(X_test)[:, 1]
        auc = roc_auc_score(Y_test, y_prob)
        if auc > best_auc:
            best_auc = auc
            best_model = lr

    y_pred = best_model.predict(X_test)
    y_prob = best_model.predict_proba(X_test)[:, 1]

    metrics_list.append({
        "Accuracy": accuracy_score(Y_test, y_pred),
        "Precision": precision_score(Y_test, y_pred),
        "Recall": recall_score(Y_test, y_pred),
        "F1": f1_score(Y_test, y_pred),
        "MCC": matthews_corrcoef(Y_test, y_pred),
        "AUC": roc_auc_score(Y_test, y_prob),
    })
results_df = pd.DataFrame(metrics_list)
summary = results_df.agg(['mean', 'std']).T
print(summary)
print(best_params_df)

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef, roc_auc_score
n_repeats = 10
metrics_list = []

for i in range(n_repeats):
    print(f"Running repetition {i+1}/{n_repeats} ...")
    X_train, X_test, Y_train, Y_test = train_test_split(
        X, y,
        test_size=0.2,
        stratify=y,
        random_state=100 + i
    )

    nb = GaussianNB()
    nb.fit(X_train, Y_train)
    y_pred = nb.predict(X_test)
    y_prob = nb.predict_proba(X_test)[:, 1]
    metrics_list.append({
        "Accuracy": accuracy_score(Y_test, y_pred),
        "Precision": precision_score(Y_test, y_pred),
        "Recall": recall_score(Y_test, y_pred),
        "F1": f1_score(Y_test, y_pred),
        "MCC": matthews_corrcoef(Y_test, y_pred),
        "AUC": roc_auc_score(Y_test, y_prob),
    })

results_df = pd.DataFrame(metrics_list)
summary = results_df.agg(['mean', 'std']).T
print(summary)
print(best_params_df)

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, ParameterGrid
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef, roc_auc_score
param_grid = {"C": [0.1, 1, 10],"kernel": ["linear", "rbf"],"gamma": ["scale", "auto"]}
grid = list(ParameterGrid(param_grid))
n_repeats = 10
metrics_list = []
best_params_list = []

for i in range(n_repeats):
    print(f"Running repetition {i+1}/{n_repeats} ...")
    X_train, X_test, Y_train, Y_test = train_test_split(
        X, y,
        test_size=0.2,
        stratify=y,
        random_state=100 + i
    )

    best_auc = -1
    best_model = None
    best_params = None

    for params in grid:
        svc = SVC(probability=True, **params)
        svc.fit(X_train, Y_train)
        y_prob = svc.predict_proba(X_test)[:, 1]
        auc = roc_auc_score(Y_test, y_prob)

        if auc > best_auc:
            best_auc = auc
            best_model = svc
            best_params = params

    y_pred = best_model.predict(X_test)
    y_prob = best_model.predict_proba(X_test)[:, 1]

    metrics_list.append({
        "Accuracy": accuracy_score(Y_test, y_pred),
        "Precision": precision_score(Y_test, y_pred),
        "Recall": recall_score(Y_test, y_pred),
        "F1": f1_score(Y_test, y_pred),
        "MCC": matthews_corrcoef(Y_test, y_pred),
        "AUC": roc_auc_score(Y_test, y_prob),
    })
    best_params_list.append(best_params)
results_df = pd.DataFrame(metrics_list)
summary = results_df.agg(['mean', 'std']).T
best_params_df = pd.DataFrame(best_params_list)

print(summary)
print(best_params_df)


In [ ]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split, ParameterGrid
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef, roc_auc_score
param_grid = {"n_estimators": [100, 200],"max_depth": [5, 10, -1],"learning_rate": [0.01, 0.1], "num_leaves": [31, 50]}
grid = list(ParameterGrid(param_grid))
n_repeats = 10
metrics_list = []
best_params_list = []
for i in range(n_repeats):
    print(f"\n--- Repetition {i+1}/{n_repeats} ---")
    X_train, X_test, Y_train, Y_test = train_test_split(
        X, y,
        test_size=0.2,
        stratify=y,
        random_state=100+i
    )

    best_auc = -1
    best_model = None
    best_params = None
    for params in grid:
        model = LGBMClassifier(**params)
        model.fit(X_train, Y_train)
        y_prob = model.predict_proba(X_test)[:, 1]
        auc = roc_auc_score(Y_test, y_prob)

        if auc > best_auc:
            best_auc = auc
            best_model = model
            best_params = params

    y_pred = best_model.predict(X_test)
    y_prob = best_model.predict_proba(X_test)[:, 1]

    metrics_list.append({
        "Accuracy": accuracy_score(Y_test, y_pred),
        "Precision": precision_score(Y_test, y_pred),
        "Recall": recall_score(Y_test, y_pred),
        "F1": f1_score(Y_test, y_pred),
        "MCC": matthews_corrcoef(Y_test, y_pred),
        "AUC": roc_auc_score(Y_test, y_prob)
    })

    best_params_list.append(best_params)
results_df = pd.DataFrame(metrics_list)
summary = results_df.agg(['mean', 'std']).T
best_params_df = pd.DataFrame(best_params_list)
print(summary)
print(best_params_df)


In [ ]:
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF, Matern, RationalQuadratic
from sklearn.model_selection import train_test_split, ParameterGrid
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef, roc_auc_score
param_grid = {"kernel": [RBF()],"max_iter_predict": [100, 200]}
grid = list(ParameterGrid(param_grid))
n_repeats = 10
metrics_list = []
best_params_list = []

for i in range(n_repeats):
    print(f"\n--- Repetition {i+1}/{n_repeats} ---")
    X_train, X_test, Y_train, Y_test = train_test_split(
        X, y,
        test_size=0.2,
        stratify=y,
        random_state=100+i
    )

    best_auc = -1
    best_model = None
    best_params = None

    for params in grid:
        gpc = GaussianProcessClassifier(**params)
        gpc.fit(X_train, Y_train)
        y_prob = gpc.predict_proba(X_test)[:, 1]
        auc = roc_auc_score(Y_test, y_prob)

        if auc > best_auc:
            best_auc = auc
            best_model = gpc
            best_params = params

    y_pred = best_model.predict(X_test)
    y_prob = best_model.predict_proba(X_test)[:, 1]

    metrics_list.append({
        "Accuracy": accuracy_score(Y_test, y_pred),
        "Precision": precision_score(Y_test, y_pred),
        "Recall": recall_score(Y_test, y_pred),
        "F1": f1_score(Y_test, y_pred),
        "MCC": matthews_corrcoef(Y_test, y_pred),
        "AUC": roc_auc_score(Y_test, y_prob)
    })

    best_params_list.append(best_params)
results_df = pd.DataFrame(metrics_list)
summary = results_df.agg(['mean', 'std']).T
best_params_df = pd.DataFrame(best_params_list)
print(summary)
print(best_params_df)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split, ParameterGrid
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef, roc_auc_score
param_grid = {"n_estimators": [100, 200],"learning_rate": [0.01, 0.1],"max_depth": [3, 5],"subsample": [0.8, 1.0]}
grid = list(ParameterGrid(param_grid))
n_repeats = 10
metrics_list = []
best_params_list = []

for i in range(n_repeats):
    print(f"\n--- Repetition {i+1}/{n_repeats} ---")
    X_train, X_test, Y_train, Y_test = train_test_split(
        X, y,
        test_size=0.2,
        stratify=y,
        random_state=100+i
    )

    best_auc = -1
    best_model = None
    best_params = None
    for params in grid:
        gb = GradientBoostingClassifier(**params)
        gb.fit(X_train, Y_train)
        y_prob = gb.predict_proba(X_test)[:, 1]
        auc = roc_auc_score(Y_test, y_prob)

        if auc > best_auc:
            best_auc = auc
            best_model = gb
            best_params = params
    y_pred = best_model.predict(X_test)
    y_prob = best_model.predict_proba(X_test)[:, 1]

    metrics_list.append({
        "Accuracy": accuracy_score(Y_test, y_pred),
        "Precision": precision_score(Y_test, y_pred),
        "Recall": recall_score(Y_test, y_pred),
        "F1": f1_score(Y_test, y_pred),
        "MCC": matthews_corrcoef(Y_test, y_pred),
        "AUC": roc_auc_score(Y_test, y_prob)
    })
    best_params_list.append(best_params)
results_df = pd.DataFrame(metrics_list)
summary = results_df.agg(['mean', 'std']).T
best_params_df = pd.DataFrame(best_params_list)
print(summary)
print(best_params_df)


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, ParameterGrid
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef, roc_auc_score
param_grid = {"C": [0.01, 0.1, 1, 10],"penalty": ["l2"],"solver": ["lbfgs"]}
grid = list(ParameterGrid(param_grid))
n_repeats = 10
metrics_list = []
for i in range(n_repeats):
    print(f"Running repetition {i+1}/{n_repeats} ...")
    X_train, X_test, Y_train, Y_test = train_test_split(
        X, y,
        test_size=0.2,
        stratify=y,
        random_state=100 + i
    )

    best_auc = -1
    best_model = None
    for params in grid:
        lr = LogisticRegression(**params, max_iter=1000)
        lr.fit(X_train, Y_train)
        y_prob = lr.predict_proba(X_test)[:, 1]
        auc = roc_auc_score(Y_test, y_prob)

        if auc > best_auc:
            best_auc = auc
            best_model = lr
    y_pred = best_model.predict(X_test)
    y_prob = best_model.predict_proba(X_test)[:, 1]

    metrics_list.append({
        "Accuracy": accuracy_score(Y_test, y_pred),
        "Precision": precision_score(Y_test, y_pred),
        "Recall": recall_score(Y_test, y_pred),
        "F1": f1_score(Y_test, y_pred),
        "MCC": matthews_corrcoef(Y_test, y_pred),
        "AUC": roc_auc_score(Y_test, y_prob),
    })
results_df = pd.DataFrame(metrics_list)
summary = results_df.agg(['mean', 'std']).T
print(summary)
print(best_params_df)

In [ ]:
import pandas as pd
import numpy as np
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split, ParameterGrid
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef, roc_auc_score
param_grid = {"n_estimators": [100, 200],"max_depth": [5, 10, -1],"learning_rate": [0.01, 0.1], "num_leaves": [31, 50]}
grid = list(ParameterGrid(param_grid))
n_repeats = 10
metrics_list = []
best_params_list = []
for i in range(n_repeats):
    print(f"\n--- Repetition {i+1}/{n_repeats} ---")
    X_train, X_test, Y_train, Y_test = train_test_split(
        X, y,
        test_size=0.2,
        stratify=y,
        random_state=100+i
    )

    best_auc = -1
    best_model = None
    best_params = None
    for params in grid:
        model = LGBMClassifier(**params)
        model.fit(X_train, Y_train)
        y_prob = model.predict_proba(X_test)[:, 1]
        auc = roc_auc_score(Y_test, y_prob)

        if auc > best_auc:
            best_auc = auc
            best_model = model
            best_params = params

    y_pred = best_model.predict(X_test)
    y_prob = best_model.predict_proba(X_test)[:, 1]

    metrics_list.append({
        "Accuracy": accuracy_score(Y_test, y_pred),
        "Precision": precision_score(Y_test, y_pred),
        "Recall": recall_score(Y_test, y_pred),
        "F1": f1_score(Y_test, y_pred),
        "MCC": matthews_corrcoef(Y_test, y_pred),
        "AUC": roc_auc_score(Y_test, y_prob)
    })

    best_params_list.append(best_params)
results_df = pd.DataFrame(metrics_list)
summary = results_df.agg(['mean', 'std']).T
best_params_df = pd.DataFrame(best_params_list)
print(summary)
print(best_params_df)


In [ ]:
from sklearn.metrics import confusion_matrix
from lightgbm import LGBMClassifier
cm_list = []
for i in range(10):
    X_train, X_test, Y_train, Y_test = train_test_split(
        X, y,
        test_size=0.2,
        stratify=y,
        random_state=100+i
    )

    model = LGBMClassifier(learning_rate=0.1,max_depth=10,n_estimators=200,num_leaves=50)
    )

    model.fit(X_train, Y_train)
    y_pred = model.predict(X_test)

    cm = confusion_matrix(Y_test, y_pred)
    cm_list.append(cm)
cm_array = np.array(cm_list)
cm_mean = cm_array.mean(axis=0)
cm_df = pd.DataFrame(cm_mean,
                     index=["Actual Inactive", "Actual Active"],
                     columns=["Predicted Inactive", "Predicted Active"])
cm_df.to_csv("LGBM_Confusion_Matrix.csv")
print("Mean Confusion Matrix Across 10 Repeats:\n", cm_mean)


In [ ]:
train = pd.concat([pd.DataFrame(X_train), Y_train.reset_index(drop=True)], axis=1)
test = pd.concat([pd.DataFrame(X_test), Y_test.reset_index(drop=True)], axis=1)

train['model'] = "Train"
test['model'] = "Test"

In [ ]:
train['model'] = "Train"
test['model'] = "Test"

In [ ]:
frames = [train,test]
pca = pd.concat(frames)
pca.head(2)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import MaxNLocator
from pandas.plotting import scatter_matrix # Changed the import statement to use pandas.plotting
# Plot the Figures Inline
%matplotlib inline


def PCA_plot(data):
    # PCA's components graphed in 2D
    # Apply Scaling
    from sklearn.decomposition import PCA
    from sklearn.preprocessing import StandardScaler
    from matplotlib import pyplot as plt
    data_pca = data

    # Apply Scaling
    X = data_pca.drop('model', axis=1).values # Use .values instead of as_matrix()
    y = data_pca['model'].values

    # Formatting
    target_names = ['Train','Test']
    colors = ['#feb236', '#6b5b95']

    # 2 Components PCA

    fig=plt.figure(2, figsize=(8, 6)) # Adjust figure size if needed

    pca = PCA(n_components=2)
    X_std = StandardScaler().fit_transform(X)
    X_r = pca.fit_transform(X_std)

    for color, i, target_name in zip(colors, ['Train','Test'], target_names):
        plt.scatter(X_r[y == i, 0], X_r[y == i, 1],
                    color=color,

                    label=target_name)
    plt.xlabel('PC1',  fontstyle= "normal", fontsize=14, fontweight='bold')
    plt.ylabel('PC2',  fontstyle= "normal", fontsize=14, fontweight='bold')


    plt.grid(False) #remove grid in 2D plot

    # Add outline around axes
    ax = plt.gca()
    for spine in ax.spines.values():
        spine.set_linewidth(1)
        spine.set_color('black')

    # Legend with box and black outline
    legend = plt.legend(loc='best', scatterpoints=1)
    legend.get_frame().set_linewidth(2)
    ax.yaxis.set_major_locator(MaxNLocator(integer=True))
    plt.tick_params(direction="out", labelsize=12)
    plt.savefig('applicability_domain.svg')
PCA_plot(pca)

In [ ]:
import matplotlib.pyplot as plt
from lightgbm import LGBMClassifier
from sklearn.decomposition import PCA

# Assuming 'X_train', 'Y_train', 'X_test', 'Y_test' are your training and test data

# Train your best model (Extra Trees, for example)
best_model = LGBMClassifier()
best_model.fit(X_train, Y_train)

# Applying PCA to training data
pca_train = PCA(n_components=2)
X_train_pca = pca_train.fit_transform(X_train)

# Applying PCA to test data
pca_test = PCA(n_components=2)
X_test_pca = pca_test.fit_transform(X_test)

# Visualize PCA of training data
plt.figure(figsize=(8, 6))
plt.scatter(X_train_pca[:, 0], X_train_pca[:, 1], color='blue', label='Training-set')

plt.xlabel('PC1')
plt.ylabel('PC2')

# Overlay PCA of test data on the same plot
plt.scatter(X_test_pca[:, 0], X_test_pca[:, 1], color='red', label='Test-set')
plt.legend()

plt.show()
plt.savefig('PCA.svg')


In [ ]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns # Import seaborn

# Assuming df_final is your DataFrame
features = ['MW', 'LogP', 'NumHDonors', 'NumHAcceptors', 'NumRotatableBonds', 'TPSA']

# Separate features and labels
X = df_final[features]
y = df_final['bioactivity_class']

# Standardize the data (important for PCA)
X_std = StandardScaler().fit_transform(X)

# Perform PCA
pca = PCA(n_components=2)  # You can change the number of components as needed
principal_components = pca.fit_transform(X_std)

# Create a new DataFrame with the principal components and relevant columns
pca_df = pd.DataFrame(data=principal_components, columns=['PC1', 'PC2'])
# Ensure the 'bioactivity_class' and 'pIC50' columns are also in pca_df
pca_df['bioactivity_class'] = df_final['bioactivity_class'].values
pca_df['pIC50'] = df_final['pIC50'].values

# Diagnostic print statement (optional)
print("Unique values in 'bioactivity_class' column:", pca_df['bioactivity_class'].unique())

# Plot the PCA results
plt.figure(figsize=(8, 6))

# Define the colors for each bioactivity class
colors = {'active': '#6b5b95', 'inactive': '#feb236'}

for bioactivity_class, color in colors.items():
    # Filter the data for the current bioactivity class
    class_data = pca_df[pca_df['bioactivity_class'] == bioactivity_class]
    # Plot the data for this class with the specified color
    if not class_data.empty:
        plt.scatter(class_data['PC1'], class_data['PC2'], alpha=0.4, color=color, label=bioactivity_class, edgecolor='black')
    else:
        print(f"Warning: No data found for bioactivity_class '{bioactivity_class}' to plot.")


plt.xlabel('PC1', fontsize=14, fontweight='bold')
plt.ylabel('PC2', fontsize=14, fontweight='bold')
plt.legend()

# --- Add this line to save the figure before showing it ---
plt.savefig('PCA.svg', format='svg')

# Show the plot
plt.show()

In [ ]:
import pandas as pd
import scipy.stats

# Assuming 'df_final' is your DataFrame containing the relevant columns

# List of columns
columns_of_interest = ["MW", "LogP", "NumHAcceptors", "NumHDonors", "NumRotatableBonds", "TPSA"]

# Dictionary to store results
summary_statistics = {}

# Separate data for group1 and group2
for group_name in df_final['bioactivity_class'].unique():
    summary_df = df_final[df_final['bioactivity_class'] == group_name][columns_of_interest].describe().transpose()

    # Calculate skewness and kurtosis
    skewness_values = df_final[df_final['bioactivity_class'] == group_name][columns_of_interest].skew()
    kurtosis_values = df_final[df_final['bioactivity_class'] == group_name][columns_of_interest].kurt()

    # Add skewness and kurtosis to the summary DataFrame
    summary_df['skew'] = skewness_values
    summary_df['kurt'] = kurtosis_values

    # Add p-values to the table
    p_values = []
    for column in columns_of_interest:
        _, p_value = scipy.stats.ttest_ind(
            df_final[df_final['bioactivity_class'] == 'active'][column].dropna(),
            df_final[df_final['bioactivity_class'] == 'inactive'][column].dropna()
        )
        p_values.append(p_value)

    # Append p-values to the summary DataFrame
    summary_df['p-value'] = p_values

    # Store the summary DataFrame in the dictionary
    summary_statistics[group_name] = summary_df

# Display the results
for group_name, stats_df in summary_statistics.items():
    print(f"\nSummary statistics for {group_name}:")
    print(stats_df[['min', 'max', '50%', 'mean', 'skew', 'kurt', 'p-value']])